# Préambule

## Introduction & Contexte

Ce projet a été réalisé par 3 étudiants de Master 1 à l'ENSAE, dans le cadre du cours "Python for Data Science". L'idée de ce projet vient d'une intuition que nous avons eu selon laquelle il était possible de prédire le succès d'un jeu vidéo auprès des utilisateurs à partir de certaines de ses caractéristiques. L'objectif principal de ce travail est donc de tenter de vérifier ou infirmer cette intuition à l'aide de techniques statistiques de traitement de données. 

De plus, nous voulions y incorporer une partie plus originale (traitement d'images...)

# Importation des modules 

## Modules de webscrapping

In [1]:
# là on importe les packages
import requests 
import urllib
import bs4

## Importation des modules de Visualisation & Modélisation des données

In [2]:
#Là on importe les autres packages
import numpy as np
import pandas as pd

In [3]:
pip show numpy

Name: numpy
Version: 1.20.3
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD
Location: /opt/anaconda3/lib/python3.9/site-packages
Requires: 
Required-by: tifffile, tables, statsmodels, seaborn, scipy, scikit-learn, scikit-image, PyWavelets, pyerfa, patsy, pandas, numexpr, numba, mkl-random, mkl-fft, matplotlib, imageio, imagecodecs, h5py, daal4py, Bottleneck, bokeh, bkcharts, astropy
Note: you may need to restart the kernel to use updated packages.


# I/ Récupération des données

## I/A Webscrapping des titres des jeux vidéos sur Wikipédia

A partir du site Wikipédia, nous allons récupérer les titres de tous les jeux vidéos sortis depuis l'année 2000. Par exemple, depuis l'URL : https://en.wikipedia.org/wiki/Category:2023_video_games nous avons accès à la liste de la quasi-totalité des jeux vidéos sortis en 2023. Nous allons donc modifier l'URL pour chaque année et constituer une liste (Liste_VG)

In [21]:
#response = requests.get(url="https://en.wikipedia.org/wiki/Category:2022_video_games", params = {"title" : "Dome Keeper"})
requests.get(url="https://en.wikipedia.org/wiki/Category:2022_video_games")
soup = bs4.BeautifulSoup(response.content, 'html.parser')
title = soup.find(id="firstHeading")
print(title.string)

None


In [23]:
print(title)

<h1 class="firstHeading mw-first-heading" id="firstHeading"><span class="mw-page-title-namespace">Category</span><span class="mw-page-title-separator">:</span><span class="mw-page-title-main">2022 video games</span></h1>


## I/B Récupération des notes des jeux

L'objectif de cette partie est d'extraire du site "Métacritic", un site spécialisé qui répertorie les avis de professionnels et d'utilisateurs sur les nouvelles sorties jeux vidéos. Pour chaque jeu dans la base de données du site, une note sur 100 est attribuée (moyenne des reviews de sites spécialisés jeux vidéos) et une note sur 10 attribuée par les utilisateurs du site. Nous allons ici récupérer pour chaque jeu ses notes Métacritic qui deviendront les variables que nous tenteront de prédire par la suite. 

On remarque que l'on peut atteindre la review du jeu à partir de l'URL, il suffit d'indiquer le titre du jeu dans l'URL, en faisant attention aux espaces. 

In [75]:
#URL du site : https://www.metacritic.com/game/
#URL d'une recherche : https://www.metacritic.com/search/gran%20theft%20auto/
#URL d'une review : https://www.metacritic.com/game/grand-theft-auto-v/
#URL d'une review d'un autre site spécialisé : https://www.ign.com/games/grand-theft-auto-v

#Base URL de toutes les reviews
URL = "https://www.metacritic.com/game/"
URLbis = "https://www.metacritic.com/game/grand-theft-auto-v/"
user_agent = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'}
site = requests.get(URLbis, user_agent)

print(site.status_code)

403


In [18]:
print(site.text)

<!DOCTYPE html><html><head><title>IGN Error 403 - Unavailable (IFW-U01)</title><link href="https://fonts.googleapis.com/css?family=Montserrat:400,700,800" rel="stylesheet" /><link rel="StyleSheet" href="https://synthetics.ign.com/css/error_page.css?bg=F06449" type="text/css" /><meta name="viewport" content="width=device-width, initial-scale=1.0"><script type="text/javascript">function qg(){var h=Math.floor(Math.random()*quotes.length)+0;document.getElementById("qb").innerHTML=quotes[h];} var quotes=["<h2>&ldquo;Aren't you a little short for a stormtrooper?&rdquo;</h2><h3>&mdash;Leia Organa</h3>","<h2>&ldquo;I am altering the deal, pray I do not alter it further.&rdquo;</h2><h3>&mdash;Darth Vader</h3>","<h2>&ldquo;No! I don't think he likes you at all... No, I don't like you either.&rdquo;</h2><h3>&mdash;C-3PO</h3>","<h2>&ldquo;I'm Luke Skywalker and I'm here to rescue you!&rdquo;</h2><h3>&mdash;Luke Skywalker</h3>","<h2>&ldquo;I recognized your foul stench when I was brought on board.&

In [23]:
pip install requests-html

     |████████████████████████████████| 83 kB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 4.5 MB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1271 sha256=7e53fdcf6123f70b8296683c49cf10c00437d214d96991a0d5884f4fb55d3ceb
  Stored in directory: /Users/tomrossa/Library/Caches/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [203]:
from requests_html import HTMLSession
session = HTMLSession()
DOM_Html = session.get(URLbis)

In [231]:
classe_note = DOM_Html.html.find('body > div > div > div.c-layoutDefault > div.c-layoutDefault_page > div.c-pageProductGame > div > div.c-productHero > div.c-productHero_container > div.c-productHero_player-scoreInfo > div.c-productHero_score-container > div.c-productHero_scoreInfo > div > div.c-productScoreInfo > div.c-productScoreInfo_scoreContent > div.c-productScoreInfo_scoreNumber > div.c-siteReviewScore_background > div', first = True)
#links = response.html.find('.c-layoutDefault.c-layoutDefault_page')
#links = response.html.find('.c-productScoreInfo_scoreNumber')
#dernier chemin : > div.c-siteReviewScore

In [236]:
print(classe_note.text)

97
User Score Generally Favorable Based on 8,886 User Ratings
8.4
My Score
Hover and click to give a rating
Saved
Add My Review
Where to Buy
Best Buy ($14.99)
All Buy Options
Latest Critic Reviews
Latest Critic Reviews
View All
Latest User Reviews
Latest User Reviews
View All
Where to Buy
Best Buy ($14.99)
All Buy Options
All Platforms
PlayStation 3
Based on 50 Critic Reviews
97
Xbox 360
Based on 58 Critic Reviews
97
PlayStation 4
Based on 66 Critic Reviews
97
Xbox One
Based on 14 Critic Reviews
97
PC
Based on 57 Critic Reviews
96
PlayStation 5
Based on 25 Critic Reviews
81
Xbox Series X
Based on 11 Critic Reviews
79
View All Platforms
Critic Reviews
Critic Reviews
View All
Metascore
Universal Acclaim Based on 66 Critic Reviews
97
100% Positive
66 Reviews
0% Mixed
0 Reviews
0% Negative
0 Reviews
All Reviews
Positive Reviews
Mixed Reviews
Negative Reviews
Critic Reviews
Critic Reviews
View All
Metascore
Universal Acclaim Based on 66 Critic Reviews
97
100% Positive
66 Reviews
0% Mixed
0 

In [116]:
a = session.get("https://fr.wikipedia.org/wiki/Alfred_Melot").html.find('body > div.vector-header-container')

In [117]:
print(a)

[<Element 'div' class=('vector-header-container',)>]


In [246]:
DOM_Html1 = session.get('https://www.metacritic.com/game/assassins-creed-2007/')
DOM_Html2 = session.get('https://www.metacritic.com/game/red-dead-redemption-2/')
jeu1 = DOM_Html1.html.find('body > div > div > div.c-layoutDefault > div.c-layoutDefault_page > div.c-pageProductGame > div > div.c-productHero > div.c-productHero_container > div.c-productHero_player-scoreInfo > div.c-productHero_score-container > div.c-productHero_scoreInfo > div > div.c-productScoreInfo > div.c-productScoreInfo_scoreContent > div.c-productScoreInfo_scoreNumber > div.c-siteReviewScore_background > div', first = True).text

print(jeu1[0 : 2])

81


In [239]:
print(DOM_Html2.html.find('body > div > div > div.c-layoutDefault > div.c-layoutDefault_page > div.c-pageProductGame > div > div.c-productHero > div.c-productHero_container > div.c-productHero_player-scoreInfo > div.c-productHero_score-container > div.c-productHero_scoreInfo > div > div.c-productScoreInfo > div.c-productScoreInfo_scoreContent > div.c-productScoreInfo_scoreNumber > div.c-siteReviewScore_background > div', first = True).text)

97
User Score Generally Favorable Based on 22,875 User Ratings
8.7
My Score
Hover and click to give a rating
Saved
Add My Review
Where to Buy
Best Buy ($41.99)
All Buy Options
Latest Critic Reviews
Latest Critic Reviews
View All
Latest User Reviews
Latest User Reviews
View All
Where to Buy
Best Buy ($41.99)
All Buy Options
All Platforms
Xbox One
Based on 34 Critic Reviews
97
PlayStation 4
Based on 109 Critic Reviews
97
PC
Based on 49 Critic Reviews
93
Critic Reviews
Critic Reviews
View All
Metascore
Universal Acclaim Based on 109 Critic Reviews
97
90% Positive
98 Reviews
1% Mixed
1 Review
0% Negative
0 Reviews
All Reviews
Positive Reviews
Mixed Reviews
Negative Reviews
Critic Reviews
Critic Reviews
View All
Metascore
Universal Acclaim Based on 109 Critic Reviews
97
90% Positive
98 Reviews
1% Mixed
1 Review
0% Negative
0 Reviews
All Reviews
Positive Reviews
Mixed Reviews
Negative Reviews
100
LEVEL (Czech Republic)
Dec 31, 2018
Amazing not only for Western lovers. No other game has proba